In [2]:
import pandas as pd
import numpy as np
import os
from openai import OpenAI
import json
from dotenv import load_dotenv
import copy

# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

load_dotenv("/mnt/c/Users/USER/Desktop/nam/gpt/.env")
api_key = os.getenv('key')
client = OpenAI(api_key=api_key)
model = "text-embedding-3-small"

In [3]:
def normalize_l2(x):
    x = np.array(x)
    if x.ndim == 1:
        norm = np.linalg.norm(x)
        if norm == 0:
            return x
        return x / norm
    else:
        norm = np.linalg.norm(x, 2, axis=1, keepdims=True)
        return np.where(norm == 0, x, x / norm)
    
def extract_vector(response):
    embedding_series = pd.Series([i.embedding for i in response.data]).apply(lambda x : normalize_l2(x[:512]))
    return embedding_series

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_embedding(text, model=model):
    return client.embeddings.create(input = [text], model=model).data[0].embedding

def search_docs(j_df, user_query):
    embedding = get_embedding(
        user_query,
        model="text-embedding-3-small"
    )
    small_emb = normalize_l2(embedding[:512])
    j_df["similarities"] = j_df['embedded_vector'].apply(lambda x: cosine_similarity(x, small_emb))
    res = j_df.sort_values("similarities", ascending=False)
    res = res.loc[res['similarities'] > 0.45][1:6]
    return res

---
# 지난방

In [107]:
file_path = './FAQ_st/datas/jinanbang'
df = pd.read_excel(file_path + '/한난_23하 공채_FAQ_최종_230816_수정.xlsx', engine='openpyxl')
df.dropna(axis=0, inplace=True)
df = df[1:].reset_index(drop=True)
df.columns = ['순번', '구분', '질문', '답변']
df

,순번,구분,질문,답변
0,1,공통,신입직원의 신분은 어떻게 되나요?,"(일반·사회형평 및 고졸인재 분야) 8급 수습직원으로 약 3개월간의 수습기간 후, ..."
1,2,공통,모집 직급별 초임연봉은 얼마인가요?,"신입직원 초임연봉은 6급(을)은 3,450만원 수준, 7급 2,650만원 수준입니다..."
2,3,공통,입사 후 근무지역은 어디인가요?,본사를 비롯한 전국 사업소 및 지사에서 근무하게 됩니다. 근무지는 우리공사 인력운영...
3,4,공통,합숙소 제공이 되나요?,"일반적으로 주민등록상 주소가 근무지역 외이며, 근무지역 내 무주택(배우자 포함)인 ..."
4,5,공통,채용 과정에서 경력을 우대받을 수 있나요?,"전문직(회계사) 분야 외 채용에서는 경력과 관련된 자격 및 우대사항은 없으며, 경력..."
5,6,공통,채용 과정에서 석박사 학위를 우대받을 수 있나요?,신입직원 채용 과정에서는 석박사 학위 보유에 따른 우대사항은 없습니다.
6,7,공통,병역 미필자가 지원 가능한가요?,"고졸인재분야는 미필자도 지원 가능하나, 그 외 분야의 경우 병역필 또는 면제자만 지..."
7,8,공통,전형과정에서 제출하는 정보 및 서류로 나이나 성별 등이 노출될 수 있지 않나요?,전형과정에서 제출하는 정보 및 서류는 자격확인 또는 본인확인 등의 목적으로만 활용됩...
8,9,공통,현재 재학중인 학생도 지원할 수 있나요?,"고졸인재분야는 전문대 및 4년제 대학 졸업자 또는 졸업예정자 지원 불가하나, 그 외..."
9,10,공통,이전 전형 점수가 다음 전형에 반영되나요?,전형단계별 합격자 결정방식은 제로 베이스(Zero-base) 방식을 적용합니다. 자...


In [108]:
for i in df.iterrows():
    print("기준질문 :", i[1]['질문'])
    print("답변 :", i[1]['답변'])
    
    print("---"*30)

기준질문 : 신입직원의 신분은 어떻게 되나요?
답변 : (일반·사회형평 및 고졸인재 분야) 8급 수습직원으로 약 3개월간의 수습기간 후, 전환평가를 거쳐 60점 이상이면 수습 해제 후 해당 직급 6급(을), 7급으로 전환됩니다. 수습기간 동안에는 인턴사원(8급수준)의 급여(약 179만원/월)가 지급됩니다.

(전문직) 별정직(기간제, 1년) 근무 후 계약기간 종료 시점에 성과평가를 통해 정규직(일반직 4급(병) 수준)으로 전환 예정입니다.
------------------------------------------------------------------------------------------
기준질문 : 모집 직급별 초임연봉은 얼마인가요?
답변 : 신입직원 초임연봉은 6급(을)은 3,450만원 수준, 7급 2,650만원 수준입니다. (성과상여금 및 기타수당 제외, 세전기준)
------------------------------------------------------------------------------------------
기준질문 : 입사 후 근무지역은 어디인가요?
답변 : 본사를 비롯한 전국 사업소 및 지사에서 근무하게 됩니다. 근무지는 우리공사 인력운영계획에 따라 배치됩니다. 희망지역에 배치되지 않을 수 있습니다.
------------------------------------------------------------------------------------------
기준질문 : 합숙소 제공이 되나요?
답변 : 일반적으로 주민등록상 주소가 근무지역 외이며, 근무지역 내 무주택(배우자 포함)인 경우 합숙소 입주대상으로 보고 있으나, 세부 자격사항은 내규를 따릅니다.
------------------------------------------------------------------------------------------
기준질문 : 채용 과정에서 경력을 우대받을 수 있나요?
답변 : 전문직(회계사) 분야 외 채용에서는 경력과 

In [109]:
response = client.embeddings.create(
    input=list(df['질문']),
    model=model
)

embedded = extract_vector(response)
df['embedded_vector'] = embedded

In [110]:
# with open(file_path + '/df_to_json_with_vertor.json', 'w') as f:
#     json.dump(json.loads(df.to_json(orient='records',force_ascii=False)), f, ensure_ascii=False, indent=4)

with open(file_path + '/df_to_json_with_vertor.json', 'r') as f:
    json_data = json.load(f)

In [111]:
for_assistant_without_vector = copy.deepcopy(json_data)
for row in for_assistant_without_vector:
    del row['embedded_vector']

In [112]:
json_df = pd.DataFrame(json_data)
res = search_docs(json_df, '자격증 뭐 내야해?')
res

,순번,구분,질문,답변,embedded_vector,similarities
19,20,입사지원,자격증은 몇 개까지 제출 가능한가요?,"일반, 사회형평 및 고졸분야 전의 경우 공통자격 최대 3개, 직무자격 최대 2개만 ...","[0.0170525192, 0.0569952085, 0.0633097377, 0.0...",0.663534
20,21,입사지원,현재 자격증 1차까지는 합격했는데 지원 가능한가요?,"자격증은 입사지원서 접수 마감일까지 최종합격(등급)이 발표된 시험으로 한정하며, 유...","[0.0157992357, 0.0229398786, 0.0430055362, 0.0...",0.547378


In [113]:
with open(file_path + '/system_prompt.txt', 'r') as f:
    system_prompt = "".join([i for i in f])
system_prompt 

'당신은 채용시스템의 FAQ기능을 대신합니다. \n사용자의 질문에 올바른 답변을 합니다. \n기준질문과 답변이 1:1로 매칭되어있는 아래 정보를 토대로 가장 맥락이 비슷한 답변을 내놓습니다.\n\n* 사용자의 질문에 답변을 할 수 없는 상황일 때 ->\n해당 질문에 대한 적절한 답변을 찾을 수 없습니다. \n자세한 사항은 FAQ-BOARD를 참고하세요.\n\n* 위 예시 형식은 반드시, 무조건 준수해야함.\n* 만약 위 예시 형식을 따르지 않는다면 벌점을 주겠음.'

In [117]:
with open(file_path + '/first_assistant.txt', 'r') as f:
    first_assistant = "".join([i for i in f])
first_assistant 

'* 대화 예시 -> \nUSER : 연봉 얼마야?\nASSISTANT : "[2] [공통] 모집 직급별 초임연봉은 얼마인가요?"에 대한 답변을 참고하세요 :\n\n신입직원 초임연봉은 6급(을)은 3,450만원 수준, 7급 2,650만원 수준입니다. (성과상여금 및 기타수당 제외, 세전기준)\n\n* 위 대화 예시 형식은 반드시, 무조건 준수해야함.\n* 만약 위 대화 예시 형식을 따르지 않는다면 벌점을 주겠음.'

---
# 수자원공사-전문직


In [118]:
file_path = './FAQ_st/datas/sujawon_jeonmunjik'
df = pd.read_excel(file_path + '/수공_24상_전문직_FAQ 휴노 최종_240520.xlsx', engine='openpyxl')
# df.dropna(axis=0, inplace=True)
# df = df[1:].reset_index(drop=True)
df.columns = ['순번', '구분', '질문', '답변']
df

,순번,구분,질문,답변
0,1,입사지원,지원자격 요건이 있나요?,채용직급 및 선발분야별 제시하는 학위/경력 조건을 모두 충족해야합니다. 자세한 내용...
1,2,입사지원,중복 지원 가능한가요?,"1개의 선발분야를 선택해야 하며, 선발분야 중복지원 및 『2024년 방호안전직 제한..."
2,3,입사지원,입사지원 시 최종제출하면 수정이 불가능한가요?,"지원서를 최종제출 하기 전에는 수정이 가능하나, 최종제출 후에는 수정은 불가합니다...."
3,4,입사지원,"시스템 오류로 입사지원서 최종제출을 못했을 경우, 마감시간 연장이 가능한가요?","어떠한 경우에도 접수기간이 연장되지 않으니, 시간적 여유를 두고 지원서를 최종제출하..."
4,5,입사지원,입사지원 시 제출서류는 어떻게 제출하나요?,지원자격 및 우대사항 등 진위 확인을 위한 제출서류는 입사지원 시스템에서 입사 지원...
5,6,입사지원,제출서류에 별도 유효기간이 있나요?,학력 및 경력증명서는 별도의 유효기간이 없으나 재직 중인 경우 경력 기간은 경력증명...
6,7,입사지원,제출서류의 경우 방문 또는 우편 접수도 가능한가요?,입사지원서는 입사지원시스템(http://kwater.cairos.co.kr)을 통해...
7,8,입사지원,제출서류 블라인드는 어떻게 하나요?,"서류 스캔본 제출 시에 개인정보(주민등록번호 뒷자리, 주소, 학교명, 연락처, 사진..."
8,9,입사지원,입사지원 시 어떠한 서류를 제출하여야 하나요?,"입사지원자격 및 우대 사항 등의 진위 확인을 위해 학력 증빙서류, 경력증명서, 자격..."
9,10,입사지원,입사지원서 작성시 [3. 교육ㆍ경력사항]은 필수인가요?,"입사지원서 [3. 교육ㆍ경력 사항]은 필수 입력항목은 아니며, 직무관련 교육과 기타..."


In [119]:
for i in df.iterrows():
    print("기준질문 :", i[1]['질문'])
    print("답변 :", i[1]['답변'])
    
    print("---"*30)

기준질문 : 지원자격 요건이 있나요?
답변 : 채용직급 및 선발분야별 제시하는 학위/경력 조건을 모두 충족해야합니다. 자세한 내용은 공고문을 확인해주십시오.

1) 직무 자격요건(지원분야별 상이함)
▸ 학위 또는 학위+경력, 자격증+학위+경력 보유자

2) 공통자격 요건 
▸ 병역필 또는 면제자 (* 단, 2024. 7. 30.까지 전역예정자로서 전형절차에 응시 가능한 경우에 지원 가능)
▸ 공사 인사규정에 의한 채용결격사유가 없는 자
▸ 입사일(2024. 7. 31.)부터 현업 전일근무 가능자
------------------------------------------------------------------------------------------
기준질문 : 중복 지원 가능한가요?
답변 : 1개의 선발분야를 선택해야 하며, 선발분야 중복지원 및 『2024년 방호안전직 제한경쟁 채용공고』 중복지원은 불가합니다. 
------------------------------------------------------------------------------------------
기준질문 : 입사지원 시 최종제출하면 수정이 불가능한가요?
답변 : 지원서를 최종제출 하기 전에는 수정이 가능하나, 최종제출 후에는 수정은 불가합니다.
최종 제출 후 수정이 필요한 지원자는 입사 지원 마감일 14:00 전까지는 삭제가 가능하오니 삭제 후 재작성하시기 바랍니다. 
------------------------------------------------------------------------------------------
기준질문 : 시스템 오류로 입사지원서 최종제출을 못했을 경우, 마감시간 연장이 가능한가요?
답변 : 어떠한 경우에도 접수기간이 연장되지 않으니, 시간적 여유를 두고 지원서를 최종제출하시기 바랍니다.
--------------------------------------------------------------------------------------

In [120]:
response = client.embeddings.create(
    input=list(df['질문']),
    model=model
)

embedded = extract_vector(response)
df['embedded_vector'] = embedded

In [121]:
with open(file_path + '/df_to_json_with_vertor.json', 'w') as f:
    json.dump(json.loads(df.to_json(orient='records',force_ascii=False)), f, ensure_ascii=False, indent=4)

with open(file_path + '/df_to_json_with_vertor.json', 'r') as f:
    json_data = json.load(f)

In [122]:
for_assistant_without_vector = copy.deepcopy(json_data)
for row in for_assistant_without_vector:
    del row['embedded_vector']

In [123]:
json_df = pd.DataFrame(json_data)
res = search_docs(json_df, '입사지원시 연령제한이 있나요?')
res

,순번,구분,질문,답변,embedded_vector,similarities
8,9,입사지원,입사지원 시 어떠한 서류를 제출하여야 하나요?,"입사지원자격 및 우대 사항 등의 진위 확인을 위해 학력 증빙서류, 경력증명서, 자격...","[0.0088732349, 0.032306074, 0.0334620929, 0.04...",0.589034
15,16,입사지원,입사지원자격기준을 초과한 경력은 어떻게 인정되나요?,입사지원자격 기준을 초과하는 경력은 공사 사규에 따라 최대 4년 및 군경력 최대 3...,"[-0.0058191035, 0.0458008214, -0.0111637265, 0...",0.584869
4,5,입사지원,입사지원 시 제출서류는 어떻게 제출하나요?,지원자격 및 우대사항 등 진위 확인을 위한 제출서류는 입사지원 시스템에서 입사 지원...,"[0.0186942348, 0.0362960634, 0.0290016099, 0.0...",0.542273
9,10,입사지원,입사지원서 작성시 [3. 교육ㆍ경력사항]은 필수인가요?,"입사지원서 [3. 교육ㆍ경력 사항]은 필수 입력항목은 아니며, 직무관련 교육과 기타...","[0.0076361455, 0.0403845636, 0.0134173569, 0.0...",0.536024
31,32,기타,입사지원 시 자기기술서(자기소개서)는 작성하지 않나요?,자기기술서(자기소개서)는 필기전형 이후 필기전형 합격자에 한해 제출하도록 안내할 예...,"[0.0551513282, 0.0579613725, -0.0079392182, -0...",0.485071


In [124]:
with open(file_path + '/system_prompt.txt', 'r') as f:
    system_prompt = "".join([i for i in f])
system_prompt 

'당신은 채용시스템의 FAQ기능을 대신합니다. \n사용자의 질문에 올바른 답변을 합니다. \n기준질문과 답변이 1:1로 매칭되어있는 아래 정보를 토대로 가장 맥락이 비슷한 답변을 내놓습니다.\n\n* 사용자의 질문에 답변을 할 수 없는 상황일 때 ->\n해당 질문에 대한 적절한 답변을 찾을 수 없습니다. \n자세한 사항은 FAQ-BOARD를 참고하세요.\n\n* 위 예시 형식은 반드시, 무조건 준수해야함.\n* 만약 위 예시 형식을 따르지 않는다면 벌점을 주겠음.'

In [160]:
import time
def make_first_assistant(df):
    first_assistant_user_question = ""
    example_index = 0
    
    while first_assistant_user_question == "":
        sorted_data = df.iloc[example_index]
        print(example_index)
        print(f"질문 원본 : {sorted_data['질문']}")
        print(f"답변 원본 : {sorted_data['답변']}")
        print()
        time.sleep(0.3)
        first_assistant_user_question = input("그만하고싶으면 '중단'이라고 입력하세요.") 
        example_index += 1
        if first_assistant_user_question == '중단':
            break
        
    if first_assistant_user_question not in ['중단', '']:
        f_a = \
f'''* 대화 예시 -> 
----------------------------------------------------------
USER : {first_assistant_user_question}
ASSISTANT : "[{sorted_data['순번']}] [{sorted_data['구분']}] {sorted_data['질문']}"에 대한 답변을 참고하세요 :

{sorted_data['답변']}
----------------------------------------------------------
* 위 대화 예시 형식은 반드시, 무조건 준수해야함.
* 만약 위 대화 예시 형식을 따르지 않는다면 벌점을 주겠음.'''
        print("-"*80)
        print(f_a)
        return f_a

In [161]:
f_a = make_first_assistant(df)

0
질문 원본 : 지원자격 요건이 있나요?
답변 원본 : 채용직급 및 선발분야별 제시하는 학위/경력 조건을 모두 충족해야합니다. 자세한 내용은 공고문을 확인해주십시오.

1) 직무 자격요건(지원분야별 상이함)
▸ 학위 또는 학위+경력, 자격증+학위+경력 보유자

2) 공통자격 요건 
▸ 병역필 또는 면제자 (* 단, 2024. 7. 30.까지 전역예정자로서 전형절차에 응시 가능한 경우에 지원 가능)
▸ 공사 인사규정에 의한 채용결격사유가 없는 자
▸ 입사일(2024. 7. 31.)부터 현업 전일근무 가능자

1
질문 원본 : 중복 지원 가능한가요?
답변 원본 : 1개의 선발분야를 선택해야 하며, 선발분야 중복지원 및 『2024년 방호안전직 제한경쟁 채용공고』 중복지원은 불가합니다. 

2
질문 원본 : 입사지원 시 최종제출하면 수정이 불가능한가요?
답변 원본 : 지원서를 최종제출 하기 전에는 수정이 가능하나, 최종제출 후에는 수정은 불가합니다.
최종 제출 후 수정이 필요한 지원자는 입사 지원 마감일 14:00 전까지는 삭제가 가능하오니 삭제 후 재작성하시기 바랍니다. 

--------------------------------------------------------------------------------
* 대화 예시 -> 
----------------------------------------------------------
USER : 지원서 수정 돼?
ASSISTANT : "[3] [입사지원] 입사지원 시 최종제출하면 수정이 불가능한가요?"에 대한 답변을 참고하세요 :

지원서를 최종제출 하기 전에는 수정이 가능하나, 최종제출 후에는 수정은 불가합니다.
최종 제출 후 수정이 필요한 지원자는 입사 지원 마감일 14:00 전까지는 삭제가 가능하오니 삭제 후 재작성하시기 바랍니다. 
----------------------------------------------------------
* 위 대화 예시 형식은 반드시, 무조건 준수해야함.
* 만약 

In [162]:
with open(file_path + '/first_assistant.txt', 'w') as f:
    f.write(f_a)
with open(file_path + '/first_assistant.txt', 'r') as f:
    first_assistant = "".join([i for i in f])
first_assistant 

'* 대화 예시 -> \n----------------------------------------------------------\nUSER : 지원서 수정 돼?\nASSISTANT : "[3] [입사지원] 입사지원 시 최종제출하면 수정이 불가능한가요?"에 대한 답변을 참고하세요 :\n\n지원서를 최종제출 하기 전에는 수정이 가능하나, 최종제출 후에는 수정은 불가합니다.\n최종 제출 후 수정이 필요한 지원자는 입사 지원 마감일 14:00 전까지는 삭제가 가능하오니 삭제 후 재작성하시기 바랍니다. \n----------------------------------------------------------\n* 위 대화 예시 형식은 반드시, 무조건 준수해야함.\n* 만약 위 대화 예시 형식을 따르지 않는다면 벌점을 주겠음.'

In [173]:
answer_content = '''"[7] [공통] 병역 미필자가 지원 가능한가요?"에 대한 답변을 참고하세요 :

고졸인재분야는 미필자도 지원 가능하나, 그 외 분야의 경우 병역필 또는 면제자만 지원가능 합니다. 여성 지원자는 미해당에 해당하며, 해당 사실에 대한 채용 시 차별 또는 불이익은 없습니다. 입사 후 군입대 시 복무
기간은 휴직처리 되며, 휴직기간 동안의 급여는 없습니다. 다만, 복무완료 시 복무기간은 근속연수에 가산됩니다.'''

"".join(answer_content.split("에 대한 답변을 참고하세요 :")[1:]).strip()

'고졸인재분야는 미필자도 지원 가능하나, 그 외 분야의 경우 병역필 또는 면제자만 지원가능 합니다. 여성 지원자는 미해당에 해당하며, 해당 사실에 대한 채용 시 차별 또는 불이익은 없습니다. 입사 후 군입대 시 복무\n기간은 휴직처리 되며, 휴직기간 동안의 급여는 없습니다. 다만, 복무완료 시 복무기간은 근속연수에 가산됩니다.'

---
# 지난방 24ver


In [167]:
# file_path = './FAQ_st/datas/jinanbang2024'
# df = pd.read_excel(file_path + '/한난_24상 공채_FAQ_240529_최종.xlsx', engine='openpyxl', dtype=object, sheet_name=None)
# df = df['챗봇용 상세 버전']
# df.dropna(axis=0, inplace=True)
# df = df[1:].reset_index(drop=True)
# df.columns = ['순번', '구분', '질문', '답변']
# no_info_answer = df.iloc[0,-1]
# df = df[1:].reset_index(drop=True)
# print(no_info_answer)
# display(df)

In [169]:
file_path = './FAQ_st/datas/jinanbang2024'
df = pd.read_excel(file_path + '/한난_24상 공채_FAQ_240529_최종.xlsx', engine='openpyxl', dtype=object)
df.dropna(axis=0, inplace=True)
df = df[1:].reset_index(drop=True)
df.columns = ['순번', '구분', '질문', '답변']
no_info_answer = df.iloc[0,-1]
df = df[1:].reset_index(drop=True)
print(no_info_answer)
display(df)

해당 질문에 대한 적절한 답변을 찾을 수 없습니다. Q&A 게시판에 질문해주시기 바랍니다.


,순번,구분,질문,답변
0,1,공통,신입직원의 신분은 어떻게 되나요?,"채용형 인턴으로 3개월 인턴기간 후, 전환평가 결과 60점 이상이면 채용예정직급인 ..."
1,2,공통,모집 직급별 초임연봉은 얼마인가요?,"신입직원 초임연봉은 6급(을)은 3,538만원 수준, 7급 2,698만원 수준입니다..."
2,3,공통,입사 후 근무지역은 어디인가요?,전국권의 경우 본사를 비롯한 전국 사업소 및 지사에서 근무하게 됩니다. 근무지는 우...
3,4,공통,합숙소 제공이 되나요?,"일반적으로 주민등록상 주소가 근무지역 외이며, 근무지역 내 무주택(배우자 포함)인 ..."
4,5,공통,채용 과정에서 경력을 우대받을 수 있나요?,"경력과 관련된 자격 및 우대사항은 없으며, 경력관련 경험을 입사지원서에 기술할 수 ..."
5,6,공통,채용 과정에서 석박사 학위를 우대받을 수 있나요?,신입직원 채용 과정에서는 석박사 학위 보유에 따른 우대사항은 없습니다.
6,7,공통,병역 미필자가 지원 가능한가요?,"고졸인재분야는 미필자도 지원 가능하며, 그 외 분야의 경우 남성은 병역필 또는 면제..."
7,8,공통,전형과정에서 제출하는 정보 및 서류로 나이나 성별 등이 노출될 수 있지 않나요?,전형과정에서 제출하는 정보 및 서류는 자격확인 또는 본인확인 등의 목적으로만 활용됩...
8,9,공통,현재 재학중인 학생도 지원할 수 있나요?,"고졸인재분야는 전문대 및 4년제 대학 졸업자 또는 졸업예정자 지원 불가하나, 그 외..."
9,10,공통,이전 전형 점수가 다음 전형에 반영되나요?,전형단계별 합격자 결정방식은 제로 베이스(Zero-base) 방식을 적용합니다. 자...


In [146]:
for i in df.iterrows():
    print("기준질문 :", i[1]['질문'])
    print("답변 :", i[1]['답변'])
    
    print("---"*30)

기준질문 : 신입직원의 신분은 어떻게 되나요?
답변 : 채용형 인턴으로 3개월 인턴기간 후, 전환평가 결과 60점 이상이면 채용예정직급인 6급(을), 7급으로 정규직 전환됩니다. 
------------------------------------------------------------------------------------------
기준질문 : 모집 직급별 초임연봉은 얼마인가요?
답변 : 신입직원 초임연봉은 6급(을)은 3,538만원 수준, 7급 2,698만원 수준입니다. 
(성과상여금 및 기타수당 제외, 세전기준)
------------------------------------------------------------------------------------------
기준질문 : 입사 후 근무지역은 어디인가요?
답변 : 전국권의 경우 본사를 비롯한 전국 사업소 및 지사에서 근무하게 됩니다. 근무지는 우리공사 인력운영계획에 따라 배치되며, 희망지역에 배치되지 않을 수 있습니다.
지역전문사원으로 지원한 경우, 임용 후 호남권역 내 광주전남지사 8년 의무 근무를 해야합니다.
(단, 4급 승진 시 타 권역으로 전보 가능)
------------------------------------------------------------------------------------------
기준질문 : 합숙소 제공이 되나요?
답변 : 일반적으로 주민등록상 주소가 근무지역 외이며, 근무지역 내 무주택(배우자 포함)인 경우 합숙소 입주대상으로 보고 있으나, 세부 자격사항은 내규를 따릅니다.
------------------------------------------------------------------------------------------
기준질문 : 채용 과정에서 경력을 우대받을 수 있나요?
답변 : 경력과 관련된 자격 및 우대사항은 없으며, 경력관련 경험을 입사지원서에 기술할 수 있습니다. 다만, 입사 후 경력증빙서류 제출 시 공사규정에 따라 호봉

In [147]:
response = client.embeddings.create(
    input=list(df['질문']),
    model=model
)

embedded = extract_vector(response)
df['embedded_vector'] = embedded

In [150]:
# with open(file_path + '/df_to_json_with_vertor.json', 'w') as f:
#     json.dump(json.loads(df.to_json(orient='records',force_ascii=False)), f, ensure_ascii=False, indent=4)

with open(file_path + '/df_to_json_with_vertor.json', 'r') as f:
    json_data = json.load(f)

In [151]:
for_assistant_without_vector = copy.deepcopy(json_data)
for row in for_assistant_without_vector:
    del row['embedded_vector']

In [152]:
json_df = pd.DataFrame(json_data)
res = search_docs(json_df, '면접 장소 어디')
res

,순번,구분,질문,답변,embedded_vector,similarities
50,51,면접전형,면접전형 날짜 및 시간은 언제인가요?,면접전형은 ’24.07.29.(월) ~ 08.09.(금)에 진행(예정)됩니다.\n자...,"[-0.0455223304, -0.0076318201, 0.050157443, -0...",0.550651
51,52,면접전형,면접전형은 어떻게 구성되어 있나요?,집단행동관찰면접 30%와 직무역량면접(인바스켓) 30% 그리고 심층면접 40%으로 ...,"[-0.0257852948, -0.0119123665, 0.0437597151, -...",0.517524
29,30,면접전형,면접전형 세부점수를 알 수 없나요?,"면접전형의 세부 점수는 정보공개법 제9조 1항 5호에 의거, 인사관리에 관한 사항이...","[-0.0649129233, 0.0431059553, 0.0191400599, -0...",0.467067


In [153]:
import time
def make_first_assistant(df):
    first_assistant_user_question = ""
    example_index = 0
    
    while first_assistant_user_question == "":
        sorted_data = df.iloc[example_index]
        print(example_index)
        print(f"질문 원본 : {sorted_data['질문']}")
        print(f"답변 원본 : {sorted_data['답변']}")
        print()
        time.sleep(0.3)
        first_assistant_user_question = input("그만하고싶으면 '중단'이라고 입력하세요.") 
        example_index += 1
        if first_assistant_user_question == '중단':
            break


    if first_assistant_user_question not in ['중단', '']:
        f_a = \
f'''* 아래 대화 예시 형식은 반드시, 무조건 준수해야함.
* 만약 아래 대화 예시 형식을 따르지 않는다면 벌점을 주겠음.

* 대화 예시 -> 
USER : 연봉 얼마야?
ASSISTANT : "[{sorted_data['순번']}] [{sorted_data['구분']}] {sorted_data['질문']}"에 대한 답변을 참고하세요 :

{sorted_data['답변']}'''
        print("-"*80)
        print(f_a)
        return f_a


def make_system_prompt(no_info_answer):
    system_prompt = \
f'''당신은 채용시스템의 FAQ기능을 대신합니다. 
사용자의 질문에 올바른 답변을 합니다. 
기준질문과 답변이 1:1로 매칭되어있는 아래 정보를 토대로 가장 맥락이 비슷한 답변을 내놓습니다.

* 아래 예시 형식은 반드시, 무조건 준수해야함.
* 만약 아래 예시 형식을 따르지 않는다면 벌점을 주겠음.

* 사용자의 질문에 답변을 할 수 없는 상황일 때 ->
{no_info_answer}
'''

    return system_prompt

In [154]:
f_a = make_first_assistant(df)

0
질문 원본 : 신입직원의 신분은 어떻게 되나요?
답변 원본 : 채용형 인턴으로 3개월 인턴기간 후, 전환평가 결과 60점 이상이면 채용예정직급인 6급(을), 7급으로 정규직 전환됩니다. 

1
질문 원본 : 모집 직급별 초임연봉은 얼마인가요?
답변 원본 : 신입직원 초임연봉은 6급(을)은 3,538만원 수준, 7급 2,698만원 수준입니다. 
(성과상여금 및 기타수당 제외, 세전기준)

--------------------------------------------------------------------------------
* 아래 대화 예시 형식은 반드시, 무조건 준수해야함.
* 만약 아래 대화 예시 형식을 따르지 않는다면 벌점을 주겠음.

* 대화 예시 -> 
USER : 연봉 얼마야?
ASSISTANT : "[2] [공통] 모집 직급별 초임연봉은 얼마인가요?"에 대한 답변을 참고하세요 :

신입직원 초임연봉은 6급(을)은 3,538만원 수준, 7급 2,698만원 수준입니다. 
(성과상여금 및 기타수당 제외, 세전기준)


In [155]:
s_p = make_system_prompt(no_info_answer)

In [156]:
# with open(file_path + '/first_assistant.txt', 'w') as f:
#     f.write(f_a)
with open(file_path + '/first_assistant.txt', 'r') as f:
    first_assistant = "".join([i for i in f])
first_assistant 

'* 아래 대화 예시 형식은 반드시, 무조건 준수해야함.\n* 만약 아래 대화 예시 형식을 따르지 않는다면 벌점을 주겠음.\n\n* 대화 예시 -> \nUSER : 연봉 얼마야?\nASSISTANT : "[2] [공통] 모집 직급별 초임연봉은 얼마인가요?"에 대한 답변을 참고하세요 :\n\n신입직원 초임연봉은 6급(을)은 3,538만원 수준, 7급 2,698만원 수준입니다. \n(성과상여금 및 기타수당 제외, 세전기준)'

In [157]:
# with open(file_path + '/system_prompt.txt', 'w') as f:
#     f.write(s_p)
with open(file_path + '/system_prompt.txt', 'r') as f:
    system_prompt = "".join([i for i in f])
system_prompt 

'당신은 채용시스템의 FAQ기능을 대신합니다. \n사용자의 질문에 올바른 답변을 합니다. \n기준질문과 답변이 1:1로 매칭되어있는 아래 정보를 토대로 가장 맥락이 비슷한 답변을 내놓습니다.\n\n* 아래 예시 형식은 반드시, 무조건 준수해야함.\n* 만약 아래 예시 형식을 따르지 않는다면 벌점을 주겠음.\n\n* 사용자의 질문에 답변을 할 수 없는 상황일 때 ->\n해당 질문에 대한 적절한 답변을 찾을 수 없습니다. Q&A 게시판에 질문해주시기 바랍니다.\n'